In [4]:
import pandas as pd
import os
import sys
from tqdm import tqdm
import json
import shutil

json_files = []
searchDir = './metaData'

for item in os.listdir(searchDir):
    json_files.append(item)
print(json_files)

['guys.json', 'sexist.json', 'everydaysexism.json', 'metoo.json', 'bloodymen.json', 'girls.json', 'boys.json', 'manspalining.json', 'slutshaming.json', 'sexism.json']


In [10]:
texts = list()
with open(os.path.join(searchDir, 'bloodymen.json'), 'r') as f:
        distros_dict = json.load(f)
        for i in range(len(distros_dict['GraphImages'])):
            try:
                texts.append(distros_dict['GraphImages'][i]['edge_media_to_caption']['edges'][0]['node']['text'])
            except IndexError:
                continue
texts

['The debate between what is considered a stereotype and what is prejudice is involved and ongoing. Why is it acceptable to stereotype women, POC, LGBTQIA etc but the second you generalise men being shit we hear NOT ALL DAVES!!! IVE NEVER RAPED A WOMAN!!! IM A NICE GUY!! WOMEN DO THAT TOO!! Hmmm it’s like the only acceptable generalisations are for non-white, non-straight, non-men. 🤔🤔🤔 Like I’m so bored of this ignorance. I’m constantly receiving abuse, harassment and being mansplained to because of this account and it only spurs me forthwith into fighting you rape-culture-facilitators more! They call me a cunt and tell me to suck their dick whilst they tell me I’m angry and damaging their world. No Dave, you just know every woman I bring over to the dark side means one less woman willing to put up with your shit! #notalldaves #notallmen #trolls #bloodymen #fuckthepatriarchy #destroythepatriarchy #feminism #mansplaining #feminist #rude #jerks #maleprivilege #abuse #thesydneyfeminists',

In [34]:
import preprocessor as p
from langdetect import detect
cleaned_texts = list()
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.HASHTAG, p.OPT.MENTION)

for tx in texts:
    tx = p.clean(tx)
    try:
        if len(tx.split()) > 5 and detect(tx) == 'en':
            cleaned_texts.append(tx)
    except:
        continue

len(set(cleaned_texts))

140

In [40]:
all_texts = list()
for file in json_files:
    with open(os.path.join(searchDir, file), 'r') as f:
        distros_dict = json.load(f)
        i = 0
        for i in range(len(distros_dict['GraphImages'])):
            try:
                caption = p.clean(distros_dict['GraphImages'][i]['edge_media_to_caption']['edges'][0]['node']['text'])
                if len(caption.split()) > 5 and detect(caption) == 'en':
                    all_texts.append(caption)
            except:
                continue
                
all_texts = list(set(all_texts))
all_texts

['Follow for more pics and quotes . Keep following . keep supporting . Bast page on instagram ~~ line',
 'Life with long hair... 🤣 they want your hair long but they always do this to you',
 "__________________ Bloody men are like buses- You wait for about a year And as soon as one approaches your Stop You. Look. At. Them. Flashing their indicators Offering you a ride. You're trying to read the destinations. You haven't much time to decide. If you make a mistake, there's no turning back. Jump off and you'll stand there and glaze. While the cars and taxis Lorries go by. And the minute, the hours, the days. __________________ Poem by: Wendy cope __________________ Art by: Joanna _______________ Tags; _________________ join the coot fam |leader| |My brutal ass|",
 "Maybe it was a book Maybe a musician or band No matter what you like there'll be someone out there pathetic enough to judge you for it. Who are you judging? And why do you even care?",
 '(with .for.insta) ... Today is the second

In [45]:
import emoji

x = []
for w in all_texts[1].split():
    w = emoji.demojize(w)
    if ':' not in w:
        x.append(w)
s = ' '.join(x)
s

'Life with long hair... they want your hair long but they always do this to you'

In [60]:
cleaned_texts = []
score = []
for t in all_texts:
    x = []
    for w in t.split():
        w = emoji.demojize(emoji.emojize(w))
        if ':' not in w and '_' not in w:
            w = "".join(filter(lambda x: ord(x)<128, w))
            x.append(w)
    cleaned_texts.append(' '.join(x))
    score.append(0)
        
cleaned_texts

['Follow for more pics and quotes . Keep following . keep supporting . Bast page on instagram ~~ line',
 'Life with long hair... they want your hair long but they always do this to you',
 "Bloody men are like buses- You wait for about a year And as soon as one approaches your Stop You. Look. At. Them. Flashing their indicators Offering you a ride. You're trying to read the destinations. You haven't much time to decide. If you make a mistake, there's no turning back. Jump off and you'll stand there and glaze. While the cars and taxis Lorries go by. And the minute, the hours, the days. Poem Wendy cope Art Joanna Tags; join the coot fam |leader| |My brutal ass|",
 "Maybe it was a book Maybe a musician or band No matter what you like there'll be someone out there pathetic enough to judge you for it. Who are you judging? And why do you even care?",
 '(with .for.insta) ... Today is the second anniversary of the viral hashtag In the last two years so many people have stood up in their own way

In [63]:
import pandas as pd
c_dict = {'texts': cleaned_texts, 'scores': score}
pd.DataFrame(c_dict).to_csv('texts.csv', index=False)